In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm= ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [3]:
class JokeState(TypedDict):
    
    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):
    
    prompt = f'generate a joke about {state["topic"]}'
    response = llm.invoke(prompt).content
    
    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):
    
    prompt =  f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content
    
    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START,'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 ={'configurable':{'thread_id':'1'}}
workflow.invoke({'topic':'cats'}, config=config1)

{'topic': 'cats',
 'joke': "Why did the cat join the Red Cross?\n\nBecause it wanted to be a first-aid kit! (First-aid kit...first-aid...cat...you know, like, helping people who are hurt? ...Okay, I'll show myself out.)",
 'explanation': 'The joke "Why did the cat join the Red Cross? Because it wanted to be a first-aid kit!" relies on a pun, a play on words based on similar sounds but different meanings. Here\'s the breakdown:\n\n* **The Setup:** The question "Why did the cat join the Red Cross?" creates an expectation that the answer will be related to the Red Cross\'s activities, which are primarily providing aid and relief in emergencies, especially medical ones.\n\n* **The Pun:** The punchline "Because it wanted to be a first-aid kit!" is where the humor lies. It plays on the sound and association of "first-aid" with "first-aid kit."\n\n* **The Connection:**  A "first-aid kit" contains supplies to provide immediate medical assistance. The cat, by wanting to *be* one, is implying it

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'cats', 'joke': "Why did the cat join the Red Cross?\n\nBecause it wanted to be a first-aid kit! (First-aid kit...first-aid...cat...you know, like, helping people who are hurt? ...Okay, I'll show myself out.)", 'explanation': 'The joke "Why did the cat join the Red Cross? Because it wanted to be a first-aid kit!" relies on a pun, a play on words based on similar sounds but different meanings. Here\'s the breakdown:\n\n* **The Setup:** The question "Why did the cat join the Red Cross?" creates an expectation that the answer will be related to the Red Cross\'s activities, which are primarily providing aid and relief in emergencies, especially medical ones.\n\n* **The Pun:** The punchline "Because it wanted to be a first-aid kit!" is where the humor lies. It plays on the sound and association of "first-aid" with "first-aid kit."\n\n* **The Connection:**  A "first-aid kit" contains supplies to provide immediate medical assistance. The cat, by wanting to *be* 

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'cats', 'joke': "Why did the cat join the Red Cross?\n\nBecause it wanted to be a first-aid kit! (First-aid kit...first-aid...cat...you know, like, helping people who are hurt? ...Okay, I'll show myself out.)", 'explanation': 'The joke "Why did the cat join the Red Cross? Because it wanted to be a first-aid kit!" relies on a pun, a play on words based on similar sounds but different meanings. Here\'s the breakdown:\n\n* **The Setup:** The question "Why did the cat join the Red Cross?" creates an expectation that the answer will be related to the Red Cross\'s activities, which are primarily providing aid and relief in emergencies, especially medical ones.\n\n* **The Pun:** The punchline "Because it wanted to be a first-aid kit!" is where the humor lies. It plays on the sound and association of "first-aid" with "first-aid kit."\n\n* **The Connection:**  A "first-aid kit" contains supplies to provide immediate medical assistance. The cat, by wanting to *be*

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti blush?\n\nBecause it saw the meat sauce!',
 'explanation': 'This joke plays on the double meaning of the word "saw."\n\n*   **Literal Meaning:** The spaghetti literally "saw" or visually perceived the meat sauce.\n*   **Figurative Meaning:** The joke implies a more suggestive meaning of "saw," suggesting the spaghetti was embarrassed or aroused by the meat sauce. This is due to the implication that the meat sauce is somehow scandalous or suggestive.\n\nThe humor comes from the unexpected shift from a simple observation to a suggestive scenario, making the spaghetti "blush" (a physical reaction to embarrassment or shyness).'}